<a href="https://colab.research.google.com/github/bavilaa/adv_ml/blob/master/BBC_News_Category_Classification_Mini_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 3: Category Classification
#author : Bruno Avila (ba2592)




In [1]:
#Loding libraries
import pandas as pd
# Set up training and test data
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

#Keras

from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop


Using TensorFlow backend.


In [0]:
#Dataframes to store the results:
r2 = pd.DataFrame(columns=['model','loss','accuaracy'])
f1 = pd.DataFrame(columns=['model','accuracy','f1_score','precision','recall'])

In [0]:
#### FUNCTIONS #####
#Simple function to transform results from keras into one hot encoding:

def predict_one_h(model,features):
  predicted_values = model.predict(features)
  return (predicted_values == predicted_values.max(axis=1,keepdims=1)).astype(float)

#model_eval_metrics( y_test,predicted_labels,classification="TRUE")

# First function to evaluate the models:

def model_eval_metrics(y_true, y_pred,model_name="",classification="TRUE"):
     if classification=="TRUE":
        accuracy_eval = accuracy_score(y_true, y_pred)
        f1_score_eval = f1_score(y_true, y_pred,average="macro",zero_division=0)
        precision_eval = precision_score(y_true, y_pred,average="macro",zero_division=0)
        recall_eval = recall_score(y_true, y_pred,average="macro",zero_division=0)
        #mse_eval = 0
        #rmse_eval = 0
        #mae_eval = 0
        #r2_eval = 0
        metricdata = {'model': [model_name],'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval]}
                      #'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     else:
        accuracy_eval = 0
        f1_score_eval = 0
        precision_eval = 0
        recall_eval = 0
        mse_eval = mean_squared_error(y_true, y_pred)
        rmse_eval = sqrt(mean_squared_error(y_true, y_pred))
        mae_eval = mean_absolute_error(y_true, y_pred)
        r2_eval = r2_score(y_true, y_pred)
        metricdata = {'accuracy': [accuracy_eval], 'f1_score': [f1_score_eval], 'precision': [precision_eval], 'recall': [recall_eval], 'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}
        finalmetricdata = pd.DataFrame.from_dict(metricdata)
     return finalmetricdata

def evaluate_r2 (model,x_test,y_test,model_name=""):
  ev = model.evaluate(x_test,y_test)
  metricdata = {'model': [model_name],'loss': ev[0], 'accuaracy': ev[1]}
                      #'mse': [mse_eval], 'rmse': [rmse_eval], 'mae': [mae_eval], 'r2': [r2_eval]}

  finalmetricdata = pd.DataFrame.from_dict(metricdata)
  return finalmetricdata



In [0]:
#Loading data
df = pd.read_csv("https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv")

## Data Description

In [0]:
#Sampling
X = df[['text']]
y = df[['category']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
#Simple Description
y_train.describe()


,category
count,1668
unique,5
top,sport
freq,387


In [7]:
y_train.category.value_counts(normalize=True)*100

sport            23.201439
business         22.422062
politics         19.124700
tech             17.865707
entertainment    17.386091
Name: category, dtype: float64

#Data Preprocessing

In [8]:
#####

# Tokenize the data into one hot vectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 200  # We will be training on 200 samples
validation_samples = 10000  # We will be validating on 10000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X['text'].to_list()) #Transforing the column into a list and next apply the tokenizer
sequences = tokenizer.texts_to_sequences(X['text'].to_list()) # converts words in each text to each word's numeric index in tokenizer dictionary.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 29726 unique tokens.


In [0]:
data = pad_sequences(sequences, maxlen=maxlen) #Transforms the array into a 2D array, max_len limit the number of words into a 100 by element (100 numbers)
labels = np.asarray(pd.get_dummies(y))

In [10]:
#Checking shape:
print ("Data shape : " + str(data.shape))
print ("Labels length: %s" % len(data))

Data shape : (2225, 100)
Labels length: 2225


In [0]:
#Sampling
x_train, x_test, y_train, y_test = train_test_split(data, labels, random_state=42)

A. A model with an embedding layer and dense layers (but w/ no layers meant for sequential data)

B. A model using an Embedding layer with Conv1d Layers

C. A model using an Embedding layer with one sequential layer (LSTM or GRU)

D. A model using an Embedding layer with stacked sequential layers (LSTM or GRU)

E. A model using an Embedding layer with bidirectional sequential layers

F. Now retrain your best model from C, D, and E using dropout (you may need to increase epochs!).

##A) A model with an embedding layer and dense layers (but w/ no layers meant for sequential data)

In [12]:
# Let's start with a model that ignores the sequential steps that make up each observation
from keras.layers import Dense, Embedding
from keras.models import Sequential
from keras.layers import Flatten, Dense

model = Sequential()
# Specify the size of your vocabulary (i.e.-10,000 terms)
# Specify the number of features you want to extract via fitting weights to your embedding matrix.
# We also specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs 
model.add(Embedding(10000, 16, input_length=maxlen))
# After the Embedding layer, 
# our activations have shape `(samples, maxlen, 8)`.

# We flatten the 3D tensor of embeddings 
# into a 2D tensor of shape `(samples, maxlen * 8)`
model.add(Flatten())

# We add the classifier on top
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 8005      
Total params: 168,005
Trainable params: 168,005
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/15
1334/1334 [==============================] - 1s 409us/step - loss: 1.6023 - acc: 0.2541 - val_loss: 1.5913 - val_acc: 0.3263
Epoch 2/15
1334/1334 [==============================] - 0s 94us/step - loss: 1.5099 - acc: 0.7856 - val_loss: 1.5512 - val_acc: 0.5000
Epoch 3/15
1334/1334 [==============================] - 0s 91us/step - loss: 1.3688 - acc: 0.9355 - val_loss: 1.4553 - val_acc: 0.5928
Epoch 4/15
1334/1334 [==============================] - 0s 92us/step - loss: 1.1485 - acc: 0.9640 - val_loss: 1.2966 - val_acc: 0.6497
Epoch 5/15
1334/1334 [==============================] - 0s 96us/step - loss: 0.8834 - acc: 0.9700 - val_loss: 1.1190 - val_acc: 0.7036
Epoch 6/15
1334/1334 [==============================] - 0s 86us/step - loss: 0.6324 - acc: 0.9798 - val_loss: 0.9572 - val_acc: 0.7575
Epoch 7/15
1334/1334 [==============================] - 0s 89us/step - loss: 0.4305 - acc: 0.9858 - val_loss: 0.8263 - val_acc: 0.7964
Epoch 8

In [13]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding and dense"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval


557/557 [==============================] - 0s 53us/step


,model,accuracy,f1_score,precision,recall
0,Embedding and dense,0.824057,0.820624,0.830079,0.818516


## 2.  A model using Conv1d Layers

In [14]:
model = Sequential()
model.add(Embedding(10000, 16, input_length=maxlen))
model.add(layers.Conv1D(32, 7, activation='relu')) 
model.add(layers.MaxPooling1D(5)) #
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())

model.add(layers.Dense(5,activation='softmax'))


model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=16,
                    validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 94, 32)            3616      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 170,981
Trainable params: 170,981
Non-trainable params: 0
________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/15
1334/1334 [==============================] - 2s 2ms/step - loss: 1.6084 - acc: 0.2189 - val_loss: 1.6071 - val_acc: 0.2066
Epoch 2/15
1334/1334 [==============================] - 0s 250us/step - loss: 1.6020 - acc: 0.2586 - val_loss: 1.6046 - val_acc: 0.1886
Epoch 3/15
1334/1334 [==============================] - 0s 254us/step - loss: 1.5971 - acc: 0.3081 - val_loss: 1.6021 - val_acc: 0.2006
Epoch 4/15
1334/1334 [==============================] - 0s 247us/step - loss: 1.5926 - acc: 0.3418 - val_loss: 1.6009 - val_acc: 0.2305
Epoch 5/15
1334/1334 [==============================] - 0s 251us/step - loss: 1.5886 - acc: 0.3103 - val_loss: 1.5994 - val_acc: 0.2335
Epoch 6/15
1334/1334 [==============================] - 0s 252us/step - loss: 1.5844 - acc: 0.3388 - val_loss: 1.5984 - val_acc: 0.2395
Epoch 7/15
1334/1334 [==============================] - 0s 262us/step - loss: 1.5800 - acc: 0.3621 - val_loss: 1.5971 - val_acc: 0.2395
Epo

In [15]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding and Conv1d Layers"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval


557/557 [==============================] - 0s 57us/step


,model,accuracy,f1_score,precision,recall
0,Embedding and Conv1d Layers,0.32316,0.180656,0.159439,0.281784


## C. A model using an Embedding layer with one sequential layer (LSTM or GRU)

In [16]:
# Example 1: simple RNN
from keras.layers import SimpleRNN, LSTM
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

model = Sequential()
model.add(Embedding(10000, 16, input_length=maxlen))
model.add(SimpleRNN(32))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=16,
                    validation_split=0.2)

# Small training data.  Increase for model improvement

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/15
1334/1334 [==============================] - 6s 4ms/step - loss: 1.6009 - acc: 0.2534 - val_loss: 1.5860 - val_acc: 0.2754
Epoch 2/15
1334/1334 [==============================] - 6s 4ms/step - loss: 1.3847 - acc: 0.5270 - val_loss: 1.3864 - val_acc: 0.3892
Epoch 3/15
1334/1334 [==============================] - 6s 4ms/step - loss: 0.9530 - acc: 0.7931 - val_loss: 1.2589 - val_acc: 0.4940
Epoch 4/15
1334/1334 [==============================] - 6s 4ms/step - loss: 0.5638 - acc: 0.9205 - val_loss: 1.2121 - val_acc: 0.5329
Epoch 5/15
1334/1334 [==============================] - 6s 4ms/step - loss: 0.3127 - acc: 0.9738 - val_loss: 1.3225 - val_acc: 0.4641
Epoch 6/15
1334/1334 [==============================] - 6s 4ms/step - loss: 0.1559 - acc: 0.9910 - val_loss: 1.3019 - val_acc: 0.4970
Epoch 7/15
1334/1334 [==============================] - 6s 4ms/step - loss: 0.0765 - acc: 0.9970 - val_loss: 1.2844 - val_acc: 0.5299
Epoch 8/15
1334

In [17]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding and 1 Sequential"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval

557/557 [==============================] - 0s 248us/step


,model,accuracy,f1_score,precision,recall
0,Embedding and 1 Sequential,0.466786,0.438519,0.43701,0.444415


## D. A model using an Embedding layer with stacked sequential layers (LSTM or GRU)

In [18]:
# Example 2: Stacked RNN layers

model = Sequential()
model.add(Embedding(10000, 32, input_length=maxlen))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=16,
                    validation_split=0.2)



/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/15
1334/1334 [==============================] - 22s 16ms/step - loss: 1.6379 - acc: 0.2234 - val_loss: 1.6211 - val_acc: 0.1826
Epoch 2/15
1334/1334 [==============================] - 21s 16ms/step - loss: 1.2859 - acc: 0.5112 - val_loss: 1.7219 - val_acc: 0.2545
Epoch 3/15
1334/1334 [==============================] - 21s 16ms/step - loss: 0.5432 - acc: 0.8478 - val_loss: 2.2015 - val_acc: 0.2754
Epoch 4/15
1334/1334 [==============================] - 21s 16ms/step - loss: 0.1503 - acc: 0.9723 - val_loss: 2.6345 - val_acc: 0.2695
Epoch 5/15
1334/1334 [==============================] - 21s 16ms/step - loss: 0.0285 - acc: 0.9978 - val_loss: 3.0535 - val_acc: 0.2904
Epoch 6/15
1334/1334 [==============================] - 21s 16ms/step - loss: 0.0072 - acc: 0.9993 - val_loss: 3.3204 - val_acc: 0.3144
Epoch 7/15
1334/1334 [==============================] - 21s 16ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 3.9566 - val_acc: 0.2784
E

In [19]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding and stacked Sequential"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval

557/557 [==============================] - 1s 915us/step


,model,accuracy,f1_score,precision,recall
0,Embedding and stacked Sequential,0.29623,0.293517,0.292887,0.295088


## E. A model using an Embedding layer with bidirectional sequential layers

In [20]:
model = Sequential()
model.add(Embedding(10000, 16, input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/15
1334/1334 [==============================] - 15s 11ms/step - loss: 1.6020 - acc: 0.2699 - val_loss: 1.5539 - val_acc: 0.2515
Epoch 2/15
1334/1334 [==============================] - 14s 11ms/step - loss: 1.2427 - acc: 0.4528 - val_loss: 1.0480 - val_acc: 0.5269
Epoch 3/15
1334/1334 [==============================] - 14s 11ms/step - loss: 0.8552 - acc: 0.6777 - val_loss: 0.7376 - val_acc: 0.7096
Epoch 4/15
1334/1334 [==============================] - 14s 11ms/step - loss: 0.5615 - acc: 0.7706 - val_loss: 0.6165 - val_acc: 0.7455
Epoch 5/15
1334/1334 [==============================] - 14s 11ms/step - loss: 0.3987 - acc: 0.8433 - val_loss: 0.5583 - val_acc: 0.7934
Epoch 6/15
1334/1334 [==============================] - 14s 11ms/step - loss: 0.2979 - acc: 0.9250 - val_loss: 0.5551 - val_acc: 0.8174
Epoch 7/15
1334/1334 [==============================] - 14s 11ms/step - loss: 0.1982 - acc: 0.9588 - val_loss: 0.8592 - val_acc: 0.7275
E

In [21]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding bidrectional"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval

557/557 [==============================] - 1s 936us/step


,model,accuracy,f1_score,precision,recall
0,Embedding bidrectional,0.820467,0.819088,0.826374,0.817894


In [22]:
#Summary:

f1

,model,accuracy,f1_score,precision,recall
0,Embedding and dense,0.824057,0.820624,0.830079,0.818516
0,Embedding and Conv1d Layers,0.323160,0.180656,0.159439,0.281784
0,Embedding and 1 Sequential,0.466786,0.438519,0.437010,0.444415
0,Embedding and stacked Sequential,0.296230,0.293517,0.292887,0.295088
0,Embedding bidrectional,0.820467,0.819088,0.826374,0.817894


In [23]:
r2

,model,loss,accuaracy
0,Embedding and dense,0.475163,0.824057
0,Embedding and Conv1d Layers,1.579606,0.323160
0,Embedding and 1 Sequential,1.827802,0.466786
0,Embedding and stacked Sequential,7.366153,0.296230
0,Embedding bidrectional,0.963615,0.820467


# F. Now retrain your best model from C, D, and E using dropout (you may need to increase epochs!).

In [24]:
model = Sequential()
model.add(Embedding(10000, 16, input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2)))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1334 samples, validate on 334 samples
Epoch 1/30
1334/1334 [==============================] - 18s 13ms/step - loss: 0.4991 - acc: 0.8000 - val_loss: 0.4957 - val_acc: 0.8000
Epoch 2/30
1334/1334 [==============================] - 17s 12ms/step - loss: 0.4664 - acc: 0.8015 - val_loss: 0.4071 - val_acc: 0.8096
Epoch 3/30
1334/1334 [==============================] - 17s 12ms/step - loss: 0.4041 - acc: 0.8207 - val_loss: 0.3771 - val_acc: 0.8257
Epoch 4/30
1334/1334 [==============================] - 16s 12ms/step - loss: 0.3833 - acc: 0.8213 - val_loss: 0.3694 - val_acc: 0.8246
Epoch 5/30
1334/1334 [==============================] - 17s 13ms/step - loss: 0.3679 - acc: 0.8261 - val_loss: 0.3638 - val_acc: 0.8293
Epoch 6/30
1334/1334 [==============================] - 17s 13ms/step - loss: 0.3643 - acc: 0.8294 - val_loss: 0.3818 - val_acc: 0.8455
Epoch 7/30
1334/1334 [==============================] - 17s 13ms/step - loss: 0.3541 - acc: 0.8382 - val_loss: 0.3564 - val_acc: 0.8299
E

In [25]:
y_predicted = predict_one_h(model,x_test)
model_name = "Embedding bidrectional w/ dropout"
meval = model_eval_metrics(y_test,y_predicted, model_name)
f1 = f1.append(meval)
meval_r2 = evaluate_r2(model,x_test,y_test,model_name)
r2 = r2.append(meval_r2)
##
meval

557/557 [==============================] - 1s 1ms/step


,model,accuracy,f1_score,precision,recall
0,Embedding bidrectional w/ dropout,0.894075,0.891765,0.892646,0.892006


### 4) Discuss 1) which model(s) performed best and speculate about 

I evaluated the models using the r2 score and f1,precision and recall. I used both approaches because business and sport categories are over-represented and it's a multi-category classification problem.

The best-performed model was the Bidrectional model.  One reason could be , in this model words (embedded words) are read also in the opposite way, this could reveal a new set of correlations related with the "traditional approach". These interactions are not possible to be found using the traditional approach. In part f) , using dropout and increasing the number of epochs helped to improve the performance of this model, however, I believe is more related to the number of epochs.



### 2) how you might try to further improve the predictive power of your model (e.g. Glove embeddings? More layers? Combining Conv1D with LSTM layers? More LSTM hidden nodes?)

Using the same models but changing the type of embedding I will identify which embedding is the best: the default or Glove. Next I will try to improve individual model performance adding more layers if that is the case. And in the last step I will play with the number of epochs. More epochs will produce a better model, however, the training time could be a problem. I will determine this trade-off.

 

 